In [4]:
# module and file imports
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import displacement_solver
import constitutive
import mesh_gen
import stress_gauss
import quadrature
import gauss_pt_coord
import stress_nodes_dc
import patch_n_int_nodes
import spr_stress
import matplotlib.pyplot as plt


In [5]:
# ............................. Inputs Parameters................................#

#Domain geometry
domain_coord = np.array([[0, 0], [48, 44], [48, 60],[0, 44],[24, 22], [48, 52], [24, 52], [0, 22]]);

# Body force components
b = np.array([[0], [0]]);

#Traction components
q = 1/16;

T = np.array([[0, 0], [0, q], [0, 0]]);

# Young's modulus
E = 1.0;

# Poisson's ration
nu = 1/3;

# problem type (0--->plane stress, 1----->plane starin)
problem_type = 0;

#Element type used for meshing (0--->4 nodes quadrilateral, 1---->8 node quadrilateral, 2-----> 9 node quadrilateral)
el_type_q4 = 0;
# el_type_q8 = 1;
# el_type_q9 = 2;

# No. of Gauss points required for integration 
ngp2d = 1;
ngp1d = 2;

# Mesh sizes to be tested
N = [4];    #Number of mesh in one direction.



In [6]:
# Q4 elements
u_list_q4 = Parallel(n_jobs = -1 , verbose = 100)(
    delayed(displacement_solver.solve_fem)(N[i], E, nu, ngp2d, ngp1d,el_type_q4, problem_type,domain_coord, b, T)
    for i in range(len(N)));

u_q4 = [];
for i in range(len(N)):
    nx = N[i];
    ny = N[i];
    node_2d = np.arange((nx+1)*(ny+1)).reshape(nx+1, ny+1);
    # print(node_2d)
    # print(node_2d[int(ny/2), -1])
    node = node_2d[int(nx/2),-1];
    dof = 2*node+1
    u_q4.append(u_list_q4[i][dof]);
# print(u_q4);

    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    7.3s finished


In [5]:
#ms = mesh size
ms = N[0];

#u = displacements at the nodes calculated directly form FEM fromulations of mesh size "ms" (u is an 1D array)
u = u_list_q4[N.index(ms)]; 

#total number of elements
nel = ms*ms

#reshaping u into u_nodes with displacment in x-direction in first column and y-direction in the second column
u_nodes = u.reshape(((ms+1)*(ms+1), 2));
# print(u_nodes);

In [6]:
# constitutive relation matrix, calculated using the fuction "Constitutube" with input E->(Young's Modulus), nu(Poisson's raton), problem_type(plane stress or plane strain)
C = constitutive.constitutive(E, nu, problem_type)
print(C);

# mesh_obj = object created to calculate 

[[1.125 0.375 0.   ]
 [0.375 1.125 0.   ]
 [0.    0.    0.375]]


In [7]:
# mesh generation 
nx = ms #number of element in x-direction
ny = ms #number of element in y-direction

# el_type = element type specifiedc. 0----> q4, q-----> q8, 2-----> q9
el_type = 0;

#mesh_obj = object created to calculated nodal coordinates ans connectivity array using functon "connectivity" and "coord_array" 
#input nx->number of element in x-direction, xy->number of element in y-direction, domain_coord->coordinates of the corner points and mid-points of the cook's skew beam problem, el_type->element type specidied.

mesh_obj = mesh_gen.MeshGenerator(nx, ny, domain_coord.reshape(16, 1), el_type);
connect = mesh_obj.connectivity();
# print(connect)
coord = mesh_obj.coord_array();
print(len(coord))
coord_df = pd.DataFrame(coord, columns=['X', 'Y'])
# print(gauss_coords);
coord_df.to_csv('Coord/mesh_'+str(nx)+'.csv')


66049


In [8]:
#Stress calculation at gauss points
#Stress (3D array) = stress at the gauss points with sigma_xx in the first column, sigma_yy in the seond column ans sigma_xy in the third column. rows number indicates the element number. layer number indicates the gauss point number.

#strains (3-D array) = strains at the gauss points with strain_xx in the first column, strain_yy in the second column and strain_xy in the third column. rows number indicates the gauss point number.

#calculated using the function "get_elemnt_stress" from the class "stress_gauss" with the following inputs;
# i->element number
# npg2d -> number of gauss points in one direction.
# el_type -> element type sepcified.
# connect -> connectivity matrix
# coord -> nodal coordinates array
# u -> displacement at nodal coordinates
# C->constitutive relation matrix;

stress = np.zeros((nel, ngp2d*ngp2d, 3));
strains = np.zeros((nel, ngp2d*ngp2d, 3));

for i in range(nel):
    stress_i_g = np.zeros((ngp2d*ngp2d, 3));
    
    strains_i_g = np.zeros((ngp2d*ngp2d, 3));

    stress_i_g, strains_i_g = stress_gauss.get_element_stress(i, ngp2d, el_type, connect, coord, u, C);
    stress[i][:][:] = stress_i_g;
    strains[i][:][:] = strains_i_g.reshape((1,3));



In [9]:
# Gauss Point coordinates
gauss_coords = np.zeros((nel, ngp2d*ngp2d, 2));


# gp = gauss points in the master domains;
# "gp" and weights" calculated using the function "quadrature" of class "quadrature"

# inputs to the function:
# ngp2d->number of gauss points in one direction.
# gauss points co-ordinates sotes in the variable "gauss_coords" with x-coordiante in the column and y-coordiante in the second column.
# gauss coordinates are calculated using the function "gauss_coords" of class "gauss_pt_coord"ArithmeticError
#inputs ngp2d->number of gauss points in one direction, vertex_coord->nodal coordinates of the current element in for loop, gp->gauss points in the master domain. el_type-> element type specified.

gp = quadrature.quadrature(ngp2d)[0];
for i in range(nel):
    node = connect[i, :];
    vertex_coord = coord[node,:].reshape(-1);
    gauss_coords[i][:][:] = gauss_pt_coord.gauss_pts(ngp2d, vertex_coord, gp, el_type);
# print(gauss_coords)
gauss_coords = gauss_coords.reshape(gauss_coords.shape[0], -1);
# print(gauss_coords);




In [10]:
#saving stress at gauss points and coordinages of gauss points into csv file for any training or plotting purpose

#reshaping stress (3D) array into 2-D array for the case in which only one gauss point is used
stress_1gp = stress.reshape((stress.shape[0], -1));
# print(stress_1gp);

#creating pandas dataframe and saving the stress data into the folder "Data"
stress_df = pd.DataFrame(stress_1gp, columns = ['sigma_x_ref', 'sigma_y_ref', 'sigma_xy_ref']);
stress_df.to_csv('Data/res_superconv_gauss_pt_1_stress_ms_'+str(nx)+'.csv');

#creating pandas dataframe and saving the stress coordinate into the folder "Coord"
gauss_poing_coord_df = pd.DataFrame(gauss_coords, columns = ['X', 'Y']);
# print(gauss_coords);
gauss_poing_coord_df.to_csv('Coord/coord_gauss_pt_'+str(nx)+'.csv');


In [11]:
# creation of patches for the implementation of the superconvergent patch recovery(spr_stress) Technique;
# Input:- ms -> mesh size;
# Output:- n_patches -> number of patches in the domain, Patch -> 2-D array with each row representing a patch of elements (4 elements in one patch)., int_nodes->all the internal nodes in the domains

patch, n_patches, int_nodes = patch_n_int_nodes.patch_n_int_nodes(ms);
# print(int_nodes);
# print(patch);
# print(n_patches);

In [12]:
#Directly calculated stress at the nodes.
# "stress_dc" -> directly calcualted stress at the nodes from FEM.
# "strain_dc" -> directly calculated strain at nodes form FEM.
# calculated using the function "stress_dc" with the following inputs:
# connect-> connectivity matrix, coord->nodal coordinates, u->nodal displacements, nel->total number of elements for the given mesh size, el_type->elements type specified, C = constitutive relation matrix.

stress_dc, strain_dc = stress_nodes_dc.stress_dc(connect, coord, u, nel, el_type, C);
print(len(strain_dc));

66049


In [13]:
# 2-D array to store the coordinates of the nodes and corresponding stresses
nodes_coord_stress_dc = np.hstack((coord, stress_dc));

# store the array of directly calculated stress and coordinates of nodes in the folder "Overall",  subfolder "DC".
nodes_coord_stress_dc_df = pd. DataFrame(nodes_coord_stress_dc, columns = ['x', 'y', 'sigma_x_dc', 'sigma_y_dc', 'sigma_xy_dc']).round(decimals = 14);
# print(nodes_coord_stress_df);
nodes_coord_stress_dc_df.to_csv('overall/dc/dc_ms_'+str(nx)+'.csv');



In [14]:
#..............stress calculated at the nodes using the spr_stress methd..............#

stress_spr = spr_stress.spr(gauss_coords, coord, connect, stress, int_nodes, n_patches ,patch, ms);
# print(len(stress_spr));

#store the calculated spr_stress stress at the nodes for the given nodes for given mesh size "ms" in the folder "overall", and subfolder "spr"
nodes_coord_stress_spr = np.hstack((coord, stress_spr))
# print(nodes_coord_stress_spr)
nodes_coord_stress_spr_df = pd. DataFrame(nodes_coord_stress_spr, columns = ['x', 'y', 'sigma_x_spr', 'sigma_y_spr', 'sigma_xy_spr']).round(decimals = 14);
# print(nodes_coord_stress_spr_df.head());
nodes_coord_stress_spr_df.to_csv('overall/spr/spr_ms_'+str(nx)+'.csv');



In [ ]:
ape = np.zeros((len(stress_spr), 3));


# print(ape_sigma_x)
for i in range(len(stress_spr)):
    ape[i][0] = abs(stress_spr[i][0]-stress_dc[i][0]);
    ape[i][1] = abs(stress_spr[i][1]-stress_dc[i][1]);
    ape[i][2] = abs(stress_spr[i][2]-stress_dc[i][2]);

# print(ape);